# Chargement des données 

In [ ]:
from pyhive import hive
import pandas as pd
from IPython.display import display

1. Paramètres de connexion Hive 

In [ ]:
hive_host = 'localhost'
hive_port = 10000
hive_username = ' ' 
hive_password = ' '

In [ ]:
#conn = None
try:
    # Établir une connexion avec authentification LDAP
    conn = hive.Connection(
        host=hive_host,
        port=hive_port,
        username=hive_username,
        password=hive_password,
        auth='LDAP'  
    )
    print("Connecté à Hive avec succès")
except Exception as e:
    print(f"Erreur lors de la connexion à Hive: {e}")

2. Chargement de données

In [ ]:
# Créer un curseur
cursor=conn.cursor()

In [ ]:
def findAll(table) :
    # Exécuter une requête pour récupérer les données de la table "catalogue"
    query = "SELECT * FROM " + table
    cursor.execute(query)

    # Charger les résultats dans un DataFrame Pandas
    data = cursor.fetchall()

    # Récupérer les noms des colonnes
    columns = [desc[0] for desc in cursor.description] 
    df = pd.DataFrame(data, columns=columns)
    print("Select all ",table)

    display(df.head())
    
    return df

    - Catalogue

In [ ]:
df_catalogue = findAll("catalogue_co2_view")

    - Immatriculation

In [ ]:
df_immatriculation = findAll("immatriculation_co2_view")

    - Marketing 

In [ ]:
df_marketing = findAll("marketing_view")

    - Client

In [ ]:
df_client = findAll("client_view")

# 1. ANALYSE EXPLORATOIRE DES DONNEES

## Statistiques descriptives / vérification des valeurs

In [ ]:
def descriptionDataframe(table, df) :
    # Affichage des statistiques descriptives
    print("Affichage des statistiques descriptives de ", table)
    display(df.describe())
    
    # Affichage des types de données et des valeurs manquantes
    print("Affichage des types de données et des valeurs manquantes  de ", table)
    display(df.info())

    # Vérification des valeurs manquantes
    print("Vérification des valeurs manquantes  de ", table)
    display(df.isnull().sum())
    
    # Vérification des doublons
    print("Vérification des doublons  de ", table)
    duplicates = df.duplicated()
    print(f"Nombre de doublons : {duplicates.sum()}")

In [ ]:
descriptionDataframe("Catalogue",df_catalogue)
#dfCatalogue.isnull().sum()

In [ ]:
descriptionDataframe("Immatriculation",df_immatriculation)

In [ ]:
# Suppression des doublons
df_immatriculation.drop_duplicates(inplace=True)
print(f"Nombre de doublons après suppression : {df_immatriculation.duplicated().sum()}")

In [ ]:
descriptionDataframe("Client",df_client)

In [ ]:
descriptionDataframe("Marketing",df_marketing)

## Matrice de corrélation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Exclure les colonnes non numériques spécifiées
df = df_catalogue.copy()

# Calcul de la matrice de corrélation
corr_matrix = df[[
    'catalogue_co2_view.puissance', 
    'catalogue_co2_view.nbplaces', 
    'catalogue_co2_view.nbportes', 
    'catalogue_co2_view.prix', 
    'catalogue_co2_view.longueur',
    'catalogue_co2_view.bonusmalus',
    'catalogue_co2_view.rejetco2',
    'catalogue_co2_view.coutenergie'
]].corr()
display(corr_matrix)

# Visualisation de la matrice de corrélation
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Heatmap des corrélations')
plt.show()


In [ ]:
# Scatter plot of puissance vs prix
plt.figure(figsize=(15, 10))
plt.subplot(2, 3, 1)
sns.scatterplot(data=data, x='catalogue_co2_view.puissance', y='catalogue_co2_view.prix')
plt.title('Puissance vs Prix')
plt.xlabel('Puissance')
plt.ylabel('Prix')

# Scatter plot of longueur vs prix
plt.subplot(2, 3, 2)
sns.scatterplot(data=data, x='catalogue_co2_view.longueur', y='catalogue_co2_view.prix')
plt.title('Longueur vs Prix')
plt.xlabel('Longueur')
plt.ylabel('Prix')

# Scatter plot of longueur vs puissance
plt.subplot(2, 3, 3)
sns.scatterplot(data=data, x='catalogue_co2_view.longueur', y='catalogue_co2_view.puissance')
plt.title('Longueur vs Puissance')
plt.xlabel('Longueur')
plt.ylabel('Puissance')

# Histogram of rejetco2
plt.subplot(2, 3, 4)
sns.histplot(data=data, x='catalogue_co2_view.rejetco2', bins=20, kde=True)
plt.title('Distribution of Rejet CO2')
plt.xlabel('Rejet CO2')
plt.ylabel('Frequency')

# Histogram of nbportes
plt.subplot(2, 3, 5)
sns.histplot(data=data, x='catalogue_co2_view.nbportes', bins=10, kde=True)
plt.title('Distribution of Nbportes')
plt.xlabel('Nbportes')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

### Conclusion:

1. Variables importantes:

   - La puissance, le prix et la longueur sont des variables importantes car elles montrent de fortes corrélations avec plusieurs autres variables. Elles peuvent être utilisées pour distinguer les catégories de véhicules.

   - Le nombre de places, le rejet CO2, le bonus et le cout d'énergie montrent peu de corrélations significatives avec les autres variables,mais ils ne doivent pas être exclu sans une analyse plus approfondie de son impact potentiel sur la catégorisation.

   - Le rejet CO2, le bonus et le cout d'énergie sont des variables très correlés cependant leur corrélations avec avec les autres variables ne doivent pas pas être exclu alors on va considerer un seul de ces variables.

3. Variables moins pertinentes: nbportes semble avoir une distribution homogène et pourrait être moins informative pour le clustering.

4. Variables à considérer pour le Clustering: puissance, nbplaces, prix, longueur, coutenergie.

# 2. Categorisation des véhicules

In [ ]:
col_predictive = [
    'catalogue_co2_view.puissance', 
    'catalogue_co2_view.nbportes', 
    'catalogue_co2_view.prix', 
    'catalogue_co2_view.longueur',
    'catalogue_co2_view.rejetco2'
]

In [ ]:
# Sélectionner les caractéristiques pertinentes
df_catalogue_used = df_catalogue[col_predictive]
df_catalogue_used.head(5)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalisation des données
scaler = StandardScaler()
scaled_df_catalogue_used = scaler.fit_transform(df_catalogue_used)
print("done")

In [ ]:
#remettre en dataframe le tableau numpy des données catalogué mise à l'echelle pour vérifier
scaled_df_catalogue_used_to_df = pd.DataFrame(scaled_df_catalogue_used, columns=df_catalogue_used.columns)
print("done")

display(scaled_df_catalogue_used_to_df.head())

In [ ]:
from sklearn.cluster import KMeans

# Utiliser la méthode du coude pour trouver le nombre optimal de clusters
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(scaled_df_catalogue_used)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Méthode du coude')
plt.xlabel('Nombre de clusters')
plt.ylabel('WCSS')
plt.show()


D'après la methode de coude, on voit ici que le nombre optimale de cluster est aux alentours de 4

In [ ]:
# Application de k-means avec le nombre de clusters choisi
kmeans = KMeans(n_clusters=4, random_state=42)
df_catalogue['catalogue_co2_view.cluster'] = kmeans.fit_predict(scaled_df_catalogue_used)
print("done")

In [ ]:
# Calculer les caractéristiques moyennes de chaque cluster
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
cluster_labels = df_catalogue['catalogue_co2_view.cluster'].unique()
cluster_summary = pd.DataFrame(cluster_centers, columns=df_catalogue_used.columns, index=cluster_labels)

display(cluster_summary)

In [ ]:
cluster_labels = {
    0: 'Citadine',
    1: 'Luxe',
    2: 'Compacte',
    3: 'Familiale'
}
df_catalogue['catalogue_co2_view.categorie'] = df_catalogue['catalogue_co2_view.cluster'].map(cluster_labels)

# Afficher le catalogue avec les catégories assignées
df_catalogue.head()

# 3. Application des catégories de véhicules définies aux données des Immatriculations

In [ ]:
df_immatriculation.head()

In [ ]:
col_predictive_immatriculation = [
    'immatriculation_co2_view.puissance', 
    'immatriculation_co2_view.longueur', 
    'immatriculation_co2_view.nbportes', 
    'immatriculation_co2_view.prix',
    'immatriculation_co2_view.rejetco2'
]

In [ ]:
# Sélectionner les caractéristiques pertinentes pour les immatriculations
df_immatriculation_used = df_immatriculation[col_predictive_immatriculation]

df_immatriculation_used.head()

In [ ]:
# Normalisation des données
scalerImma = StandardScaler()
scaled_df_immatriculation_used = scalerImma.fit_transform(df_immatriculation_used)
print("done scaling")

In [ ]:
# Utiliser le modèle de clustering pour prédire les clusters des données d'immatriculation
df_immatriculation['immatriculation_co2_view.cluster'] = kmeans.predict(scaled_df_immatriculation_used)

# Mapper les clusters prédits aux catégories de véhicules correspondantes
cluster_labels = {
    0: 'Citadine',
    1: 'Luxe',
    2: 'Compacte',
    3: 'Familiale'
}
df_immatriculation['immatriculation_co2_view.categorie'] = df_immatriculation['immatriculation_co2_view.cluster'].map(cluster_labels)
print("done")

# Afficher le DataFrame d'immatriculation avec les catégories assignées
df_immatriculation.head()


# 4. Fusion des données Clients et Immatriculations

In [ ]:
# Fusionner les deux DataFrames sur la colonne 'immatriculation'
df_fusion = pd.merge(df_client, df_immatriculation, how='inner', left_on='client_view.immatriculation', right_on='immatriculation_co2_view.immatriculation')
print("Fusion done")

# Afficher les premières lignes du DataFrame fusionné
df_fusion.head()

In [ ]:
columns_to_drop = ["immatriculation_co2_view.immatriculation", "immatriculation_co2_view.marque", "immatriculation_co2_view.nom",
                   "immatriculation_co2_view.puissance", "immatriculation_co2_view.longueur", "immatriculation_co2_view.nbplaces",
                   "immatriculation_co2_view.nbportes", "immatriculation_co2_view.couleur", "immatriculation_co2_view.occasion",
                   "immatriculation_co2_view.prix", "immatriculation_co2_view.cluster", "immatriculation_co2_view.rejetco2",
                   "immatriculation_co2_view.bonusmalus", "immatriculation_co2_view.coutenergie", "client_view.immatriculation"]

df_fusion_new = df_fusion.drop(columns=columns_to_drop)

In [ ]:
df_fusion_new.head()

### Mettre les données de fusion entre immatriculations et clients dans hdfs

In [ ]:
df_fusion_new.to_csv('/vagrant/tpa_groupe_14/data/fusion.csv', index=False, encoding='utf-8')

In [ ]:
!hadoop fs -mkdir -p /tpa_groupe_14/data/fusion

In [ ]:
!hadoop fs -put -f /vagrant/tpa_groupe_14/data/fusion.csv /tpa_groupe_14/data/fusion

In [ ]:
!hadoop fs -ls /tpa_groupe_14/data/fusion